In [1]:
# Dependencies
import tweepy
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [2]:
# Twitter API Keys. Place your keys here.
consumer_key = "jHszemnvaELuv5AhnQR4ZLdrA"
consumer_secret = "A4RjPlgzfPzwGuLloxjSglNRvpKz1UVf0QYdnEOn77966akdgD"
access_token = "706845518274699264-7rizpUwb9IUpLKnsE8hlUee1UvOraGF"
access_token_secret = "jT1cBzMTXcsy4lt7zLW8TUHi5kAX8r7BIX8uNc8bOGndo"

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [3]:
# Target User Account
target_users = ["@CNBC","@CNNMoney", "@WSJ", "@YahooFinance", "@FinancialTimes"]

#Empty list to aggregate retrieved data for sentiments.
sentiment_time = []

for user in target_users:

# Lists for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

# A list to hold tweet timestamps
    tweet_times = []
    
# Loop through 5 pages of tweets (total 100 tweets)
    for page in tweepy.Cursor(api.user_timeline, id=user).pages(1):
        
# Get all tweets from home feed
#public_tweets = api.user_timeline(user)
        for status in page:
            tweet = json.dumps(status._json, indent=3)
            tweet = json.loads(tweet)
            text = tweet['text']
            raw_time = tweet["created_at"]
            #print(raw_time)
            tweet_times.append(raw_time)
            
            converted_times = []
            for x in tweet_times:
                clean_time = datetime.strptime(x, "%a %b %d %H:%M:%S %z %Y")
                converted_times.append(clean_time)             
            
            
            # Run Vader Analysis on each tweet
            compound = analyzer.polarity_scores(text)["compound"]
            pos = analyzer.polarity_scores(text)["pos"]
            neu = analyzer.polarity_scores(text)["neu"]
            neg = analyzer.polarity_scores(text)["neg"]
    
# Add each value to the appropriate array
            sentiment_time.append({"Tweeter": user,
                                  "Comp_Sent": compound,
                                  "Posi_Sent": pos,
                                  "Nuet_Sent": neu,
                                  "Nege_Sent": neg,
                                  "Time": clean_time})
   

In [ ]:
print(clean_time)

In [ ]:
'''# Convert tweet timestamps to datetime objects that can be manipulated by Python
converted_times = []
    for x in tweet_times:
    clean_time = datetime.strptime(x, "%a %b %d %H:%M:%S %z %Y")
    converted_times.append(clean_time)'''

In [4]:
# Convert dictionary into a dataframe.
sentiment_pd = pd.DataFrame(sentiment_time)
com = sentiment_pd["Comp_Sent"]
pos = sentiment_pd["Posi_Sent"]
neg = sentiment_pd["Nege_Sent"]
nue = sentiment_pd["Nuet_Sent"]
clean_time = sentiment_pd["Time"]

sentiment_pd.head()

,Comp_Sent,Nege_Sent,Nuet_Sent,Posi_Sent,Time,Tweeter
0,0.0000,0.000,1.000,0.000,2018-03-20 13:29:03+00:00,@CNBC
1,-0.4404,0.162,0.838,0.000,2018-03-20 13:27:05+00:00,@CNBC
2,0.0000,0.000,1.000,0.000,2018-03-20 13:18:15+00:00,@CNBC
3,0.2023,0.000,0.886,0.114,2018-03-20 13:13:34+00:00,@CNBC
4,-0.3818,0.157,0.843,0.000,2018-03-20 13:03:59+00:00,@CNBC


In [ ]:
'''# Print the Averages for each user
print("")
print("User: %s" % user)
print("Compound: %s" % (com))
#print("Positive: %s" % np.mean(positive_list))
#print("Neutral: %s" % np.mean(neutral_list))
#print("Negative: %s" % np.mean(negative_list))
#print(len(compound_list))
'''

In [5]:
# Save DF to a csv file.
sentiment_pd.to_csv('Senitment_On_Tweets.csv')

In [9]:
# Test Plot
colors = {'@CNBC':'red', '@CNNMoney':'blue', '@YahooFinance':'green', '@WSJ':'black', '@FinancialTimes': 'yellow'}
plt.scatter(t=sentiment_pd['Time'].value, com, c=sentiment_pd['Tweeter'].apply(lambda x: colors[x]),marker='o', alpha =.3, linewidths = 2)
plt.xlim(100,0)
plt.ylim(1.00, -1.00)
plt.xticks(plt.xticks()[0], [time.strftime("%m/%d/%y, %H:%M:%S", time.localtime(item)) for item in plt.xticks()[0]*3600])
plt.xticks(rotation='vertical')

SyntaxError: positional argument follows keyword argument (<ipython-input-9-1612424876a0>, line 3)

In [ ]:

plt.xlabel("100 Recent Tweets")
plt.ylabel("Total (Sentiment) Polarity")
plt.title("Business News Sentiment")
#plt.legend("")

In [ ]:
plt.show()